In [ ]:
%%bash
version = $(docker --version 2> /dev/null)

if [ -n "$version"]; then
    echo "Docker is installed: $version"
    docker pull yarrowdocker/bindcraft:latest
else
    echo "Docker is not installed. Please install Docker to proceed."
    exit 1
fi


Docker version 28.2.2, build e6534b4


In [ ]:
%%bash
mkdir -p root/bindcraft
cd root/bindcraft
pwd

In [ ]:
%%bash
git clone https://github.com/A-Yarrow/BindCraft-Cloud.git .

test output


In [ ]:
%%bash

# AF2 weight download (avoid storing in the Docker image)
# Ensure expected weights are mounted externally and accessible

params_dir="params"

mkdir -p "${params_dir}" 
cd "${params_dir}" 
wget --progress=dot:giga https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar 
tar -xvf alphafold_params_2022-12-06.tar 
rm alphafold_params_2022-12-06.tar 
cd ..

if [ ! -f "${params_dir}/params_model_5_ptm.npz" ]; then
    echo -e "Error: AlphaFold2 weights not found in ${params_dir}."
    echo -e "Please mount the weights directory to /app/bindcraft/params when running the container."
    exit 1
else
    echo -e "AlphaFold2 weights found at ${params_dir}"
fi

In [ ]:
%%bash
# Run Bindcraft docker container in background
# Add "--gpus all" when running in production 
docker run -dit --name bindcraft_session \
  -v $(pwd)/settings_target:/app/bindcraft/settings_target \
  -v $(pwd)/settings_filters:/app/bindcraft/settings_filters \
  -v $(pwd)/settings_advanced:/app/bindcraft/settings_advanced \
  -v $(pwd)/params:/app/bindcraft/params \
  bindcraft:latest bash


a42475b8b8ee30f9d59f29e2d83ef909a525d4045c378f2982bbfe8fd0be1e5c


In [ ]:

%%bash
# Install PyRosetta in the Bindcraft docker container
# This is done in the container to avoid installing it in the image.
# Before running this cell, make sure you agree to the terms of use:
# https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.PyRosetta.md

docker exec -u root bindcraft_session bash -c "
  source /opt/conda/etc/profile.d/conda.sh && \
  conda activate BindCraft && \
  mkdir -p /app/pyrosetta && \
  cd /app/pyrosetta && \
  mamba install pyrosetta -c https://conda.graylab.jhu.edu -y
  "



Pinned packages:

  - python=3.10


Transaction

  Prefix: /opt/conda/envs/BindCraft

  Updating specs:

   - pyrosetta


  Package                         Version  Build       Channel                   Size
────��──────────────────────────────────────────────────────────────────���───────────────
  Install:
─────────────��──────────────────────────────────────────────────────────────────���──────

  + pyrosetta  2025.24+release.8e1e5e54f0  py310_0     conda.graylab.jhu.edu      1GB
  + zlib                            1.3.1  hb9d3cd8_2  conda-forge               92kB

  Summary:

  Install: 2 packages

  Total download: 1GB

───────────────��──────────────────────────────────────────────────────────────────���────



Transaction starting


In [ ]:
# Run Bindcraft with the specified settings and filters
%%bash
target="PDL1.json"  # Change this to your target file name
if [ ! -f "$(pwd)/settings_target/$target" ]; then
  echo "Target file $target not found in targets directory."
  exit 1
fi
docker exec bindcraft_session bash -c "
  python /app/bindcraft/bindcraft.py \
    --settings /app/bindcraft/settings_target/$target \
    --filters /app/bindcraft/settings_filters/default_filters.json \
    --advanced /app/bindcraft/settings_advanced/default_4stage_multimer.json"

In [ ]:

import json
import ipywidgets as widgets
from IPython.display import display

# Example JSON settings dict (like what you'd have in your settings file)
settings = {
    "binder_length": 50,
    "temperature": 300,
    "use_multimer": True,
    "target_name": "PDL1"
}

# Create widgets matching those fields
binder_length_widget = widgets.IntSlider(value=settings["binder_length"], min=10, max=100, description='Binder Length')
temperature_widget = widgets.IntText(value=settings["temperature"], description='Temperature (K)')
use_multimer_widget = widgets.Checkbox(value=settings["use_multimer"], description='Use Multimer')
target_name_widget = widgets.Text(value=settings["target_name"], description='Target Name')

# Display widgets
display(binder_length_widget, temperature_widget, use_multimer_widget, target_name_widget)

# Button to save the current widget values back to JSON file
save_button = widgets.Button(description="Save Settings")

output = widgets.Output()

def save_clicked(b):
    # Update dict from widget values
    settings["binder_length"] = binder_length_widget.value
    settings["temperature"] = temperature_widget.value
    settings["use_multimer"] = use_multimer_widget.value
    settings["target_name"] = target_name_widget.value
    
    # Save to JSON file
    with open("settings_output.json", "w") as f:
        json.dump(settings, f, indent=2)
    
    with output:
        output.clear_output()
        print("Settings saved to settings_output.json")

save_button.on_click(save_clicked)

display(save_button, output)


ModuleNotFoundError: No module named 'ipywidgets'